In [ ]:
import os
os.environ['HF_TOKEN'] = ''

import webbrowser
import http.server
import socketserver
import threading
import json
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

PORT = 8001

json_results = None      # 存放推荐 API 返回 JSON
last_query = None        # 用户输入 query



# 加载Chatbot模型


print("Loading Chatbot model...")

path = "Dellalala1/LlaMA_3.2_1B_Fine-tuned"
model = AutoModelForCausalLM.from_pretrained(path, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(path)

print("Chatbot model loaded.")



# Chatbot 原函数


def generate_answer(question: str):
    prompt = f"### Instruction:\n{question}\n\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            temperature=1.1,
            top_p=0.85,
            repetition_penalty=1.1,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.split("### Response:")[-1].strip()
    return answer


def format_recommendation(df: pd.DataFrame):
    context = "Here are the top recommended products:\n\n"
    for idx, row in df.iterrows():
        context += (
            f"Product {idx + 1}:\n"
            f"- Name: {row['title']}\n"
            f"- Product ID: {row['pid']}\n"
            f"- Description: {row['summary']}\n"
            f"- Score: {row['scores']}\n\n"
        )
    return context


def format_inquiry(df: pd.DataFrame):
    context = "Here is the product information:\n\n"
    for idx, row in df.iterrows():
        context += (
            f"Product {idx + 1}:\n"
            f"- Name: {row['title']}\n"
            f"- Product ID: {row['pid']}\n"
            f"- Description: {row['summary']}\n\n"
        )
    return context


def generate_answer_with_rag(user_query, rag_results: pd.DataFrame, intent: str):
    if intent == "product recommendation":
        product_context = format_recommendation(rag_results)
        enhanced_prompt = f"""
You are a helpful customer service assistant.
Based on the following recommended products, answer the user's query.

{product_context}

User Query: {user_query}
"""
        return generate_answer(enhanced_prompt)

    elif intent == "product inquiry":
        product_context = format_inquiry(rag_results)
        enhanced_prompt = f"""
You are a helpful customer service assistant.
Based on the following product information, answer the user's question.

{product_context}

User Query: {user_query}
"""
        return generate_answer(enhanced_prompt)

    return generate_answer(user_query)



# HTML前端页面


HTML_CONTENT = r"""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Beauty Product Recommendation</title>

<style>
    body { background:#fafafa; font-family:Arial; margin:0; padding:0; }
    header { background:#4C7AF2; padding:20px; color:white; font-size:24px; text-align:center; }
    .container { width:90%; max-width:900px; margin:auto; padding:20px; }

    .section-title { font-size:20px; font-weight:bold; margin-top:20px; margin-bottom:12px; }

    .grid-row {
        display:grid;
        grid-template-columns: 140px 240px 100px;
        gap:10px;
        align-items:center;
        margin-bottom:10px;
    }

    .result-answer {
        margin-top:12px;
        padding:12px;
        background:white;
        border-radius:8px;
        box-shadow:0 1px 4px rgba(0,0,0,0.08);
        white-space:pre-wrap;
        font-size:15px;
    }
</style>

</head>

<body>
<header>Beauty Product Recommendation System</header>

<div class="container">

    <div class="section-title">Query</div>
    <textarea id="query" style="width:100%; height:70px;"></textarea>

    <div class="section-title">Constraints</div>

    <div class="grid-row">
        <label>Brand</label><input id="brand"><label><input id="hard_brand" type="checkbox">Hard</label>
    </div>
    <div class="grid-row">
        <label>Price Min</label><input id="price_min" type="number"><label><input id="hard_price_min" type="checkbox">Hard</label>
    </div>
    <div class="grid-row">
        <label>Price Max</label><input id="price_max" type="number"><label><input id="hard_price_max" type="checkbox">Hard</label>
    </div>
    <div class="grid-row">
        <label>Rating Min</label><input id="rating_min" type="number"><label><input id="hard_rating_min" type="checkbox">Hard</label>
    </div>

    <button onclick="sendRequest()">Recommend</button>

    <h3>Results</h3>
    <div id="results"></div>

</div>

<script>
async function sendRequest() {

    const payload = {
        query: document.getElementById("query").value,
        constraints: {
            brand: document.getElementById("brand").value,
            price_min: parseFloat(document.getElementById("price_min").value),
            price_max: parseFloat(document.getElementById("price_max").value),
            rating_min: parseFloat(document.getElementById("rating_min").value),
        },
        hard_flags: {
            brand: document.getElementById("hard_brand").checked,
            price_min: document.getElementById("hard_price_min").checked,
            price_max: document.getElementById("hard_price_max").checked,
            rating_min: document.getElementById("hard_rating_min").checked,
        },
        topn: 10,
        ppr_k: 30
    };

    document.getElementById("results").innerHTML =
        "<div class='result-answer'>Running recommendation...</div>";

    const recRes = await fetch("http://127.0.0.1:8000/recommend", {
        method:"POST",
        headers:{"Content-Type":"application/json"},
        body:JSON.stringify(payload)
    });

    const recJson = await recRes.json();

    const saveRes = await fetch("http://127.0.0.1:8001/save", {
        method:"POST",
        headers:{"Content-Type":"application/json"},
        body: JSON.stringify({query: payload.query, recs: recJson})
    });

    const ansJson = await saveRes.json();

    document.getElementById("results").innerHTML =
        `<div class='result-answer'>${ansJson.answer}</div>`;
}
</script>

</body>
</html>
"""



# 后端：处理 /save 调用 


class Handler(http.server.BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type","text/html; charset=utf-8")
        self.end_headers()
        self.wfile.write(HTML_CONTENT.encode("utf-8"))

    def do_POST(self):
        global json_results, last_query

        if self.path == "/save":
            length = int(self.headers.get("Content-Length",0))
            payload = json.loads(self.rfile.read(length).decode("utf-8"))

            last_query = payload["query"]
            rec_json = payload["recs"]

            print("\n===== From Frontend =====")
            print("Query:", last_query)

            results = rec_json.get("results", [])
            rag_df = pd.DataFrame(results)

            intent = "product recommendation"

            answer = generate_answer_with_rag(
                user_query=last_query,
                rag_results=rag_df,
                intent=intent
            )

            resp = {"answer": answer}
            resp_bytes = json.dumps(resp, ensure_ascii=False).encode("utf-8")

            self.send_response(200)
            self.send_header("Content-Type", "application/json; charset=utf-8")
            self.end_headers()
            self.wfile.write(resp_bytes)

        else:
            self.send_response(404)
            self.end_headers()



# 启动本地网页 

def start_server():
    with socketserver.TCPServer(("", PORT), Handler) as httpd:
        print(f"Client UI running at: http://127.0.0.1:{PORT}")
        httpd.serve_forever()

if __name__ == "__main__":
    threading.Thread(target=start_server, daemon=True).start()
    webbrowser.open(f"http://127.0.0.1:{PORT}")
    input("Press Enter to exit...\n")
